# Домашнее задание  № 5. Матричные разложения/Тематическое моделирование

### Задание № 1 (8 баллов)

Попробуйте матричные разложения с 4 классификаторами - SGDClassifier, KNeighborsClassifier,  RandomForest, ExtraTreesClassifier (про него подробнее почитайте в документации, он похож на RF). Используйте и NMF, SVD и LDA. Сравните результаты на кросс-валидации и выберите лучшее сочетание.

В итоге у вас должно получиться, как минимум 12 моделей (три разложения на каждый классификатор). Используйте 1 и те же параметры кросс-валидации. Параметры векторизации, параметры K в матричных разложениях, параметры классификаторов могут быть разными между экспериментами.

Можете взять поменьше данных, если все будет обучаться слишком долго (не ставьте параметр K слишком большим в NMF и LDA, иначе точно будет слишком долго)

In [ ]:
!pip install gensim pymorphy2 seaborn pyLDAvis razdel numpy setuptools pandas

In [19]:
import warnings
warnings.filterwarnings("ignore")

In [121]:
import gensim
import pandas as pd
import numpy as np
from pymorphy2 import MorphAnalyzer
import pyLDAvis.gensim_models
from collections import Counter
from string import punctuation
from razdel import tokenize as razdel_tokenize
from IPython.display import Image
from IPython.core.display import HTML
from sklearn.decomposition import TruncatedSVD, NMF, PCA, LatentDirichletAllocation
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.metrics.pairwise import cosine_distances
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, StratifiedKFold
from matplotlib import pyplot as plt
import seaborn as sns
morph = MorphAnalyzer()

In [5]:
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    normalized_text = [morph.parse(word)[0].normal_form for word in normalized_text]
    return ' '.join(normalized_text)

In [106]:
data = pd.read_csv('avito_category_classification.csv')

In [107]:
data['description_norm'] = data['description'].apply(normalize)

In [119]:
def eval_table(X, y, pipeline, N=6):
    # измени ф-ю с семинара, чтобы выдавался только f1-mean, т.к. мне кажется,
    # что по нему будет очень удобно выбрать лидирующее сочетание разложение+классификатор 
    labels = list(set(y))
    fold_metrics = pd.DataFrame(index=labels)   
    kfold = StratifiedKFold(n_splits=N, shuffle=True, )
    
    for i, (train_index, test_index) in enumerate(kfold.split(X, y)):
        pipeline.fit(X[train_index], y[train_index])
        preds = pipeline.predict(X[test_index])

        fold_metrics[f'f1_{i}'] = f1_score(y[test_index], preds, labels=labels, average=None)

    result = pd.DataFrame(index=labels)
    result['f1'] = fold_metrics[[f'f1_{i}' for i in range(N)]].mean(axis=1).round(2)
    result.loc['mean'] = result.mean().round(2)
    
    return result.loc['mean']['f1']

In [126]:
# Вопрос:
# Можно ли избавиться от повторногой векторизации в pipeline,
# а то получается, что я постоянно трачу по 3 минуты только на это,
# когда сам классификатор работает за секунды... Либо я чего-то не понимаю...

pipeline_SVD_RF = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), min_df=3, max_df=0.3)),
    ('decomposition', TruncatedSVD(100)),
    ('clf', RandomForestClassifier(n_estimators=100, max_depth=6))
])

pipeline_SVD_ExtraTrees = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), min_df=3, max_df=0.3)),
    ('decomposition', TruncatedSVD(100)),
    ('clf', ExtraTreesClassifier(n_estimators=100, max_depth=6))
])

pipeline_SVD_SGD = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), min_df=3, max_df=0.3)),
    ('decomposition', TruncatedSVD(100)),
    ('clf', SGDClassifier(max_iter=1000, tol=1e-3))
])

pipeline_SVD_KN = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), min_df=3, max_df=0.3)),
    ('decomposition', TruncatedSVD(100)),
    ('clf', KNeighborsClassifier(n_neighbors=3))
])

pipeline_NMF_RF = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), min_df=3, max_df=0.3)),
    ('decomposition', NMF(100)),
    ('clf', RandomForestClassifier(n_estimators=100, max_depth=6))
])

pipeline_NMF_ExtraTrees = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), min_df=3, max_df=0.3)),
    ('decomposition', NMF(100)),
    ('clf', ExtraTreesClassifier(n_estimators=100, max_depth=6))
])

pipeline_NMF_SGD = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), min_df=3, max_df=0.3)),
    ('decomposition', NMF(100)),
    ('clf', SGDClassifier(max_iter=1000, tol=1e-3))
])

pipeline_NMF_KN = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), min_df=3, max_df=0.3)),
    ('decomposition', NMF(100)),
    ('clf', KNeighborsClassifier(n_neighbors=3))
])

pipeline_LDA_RF = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), min_df=3, max_df=0.3)),
    ('decomposition', LatentDirichletAllocation(100)),
    ('clf', RandomForestClassifier(n_estimators=100, max_depth=6))
])

pipeline_LDA_ExtraTrees = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), min_df=3, max_df=0.3)),
    ('decomposition', LatentDirichletAllocation(100)),
    ('clf', ExtraTreesClassifier(n_estimators=100, max_depth=6))
])

pipeline_LDA_SGD = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), min_df=3, max_df=0.3)),
    ('decomposition', LatentDirichletAllocation(100)),
    ('clf', SGDClassifier(max_iter=1000, tol=1e-3))
])

pipeline_LDA_KN = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), min_df=3, max_df=0.3)),
    ('decomposition', LatentDirichletAllocation(100)),
    ('clf', KNeighborsClassifier(n_neighbors=3))
])

In [127]:
NMF_RF = eval_table(data['description_norm'], data['category_name'], pipeline_NMF_RF)
NMF_SGD = eval_table(data['description_norm'], data['category_name'], pipeline_NMF_SGD)
NMF_ExtraTrees = eval_table(data['description_norm'], data['category_name'], pipeline_NMF_ExtraTrees)
NMF_KN = eval_table(data['description_norm'], data['category_name'], pipeline_NMF_KN)

SVD_RF = eval_table(data['description_norm'], data['category_name'], pipeline_SVD_RF)
SVD_SGD = eval_table(data['description_norm'], data['category_name'], pipeline_SVD_SGD)
SVD_ExtraTrees = eval_table(data['description_norm'], data['category_name'], pipeline_SVD_ExtraTrees)
SVD_KN = eval_table(data['description_norm'], data['category_name'], pipeline_SVD_KN)

LDA_RF = eval_table(data['description_norm'], data['category_name'], pipeline_LDA_RF)
LDA_SGD = eval_table(data['description_norm'], data['category_name'], pipeline_LDA_SGD)
LDA_ExtraTrees = eval_table(data['description_norm'], data['category_name'], pipeline_LDA_ExtraTrees)
LDA_KN = eval_table(data['description_norm'], data['category_name'], pipeline_LDA_KN)

In [128]:
res_NMF = {'NMF_RF': NMF_RF, 'NMF_SGD': NMF_SGD, 'NMF_ExtraTrees': NMF_ExtraTrees, 'NMF_KN': NMF_KN}
NMF_DF = pd.DataFrame.from_dict(res_NMF, orient='index', columns=['F1-mean'])

res_SVD = {'SVD_RF': SVD_RF, 'SVD_SGD': SVD_SGD, 'SVD_ExtraTrees': SVD_ExtraTrees, 'SVD_KN': SVD_KN}
SVD_DF = pd.DataFrame.from_dict(res_SVD, orient='index', columns=['F1-mean'])

res_LDA = {'LDA_RF': LDA_RF, 'LDA_SGD': LDA_SGD, 'LDA_ExtraTrees': LDA_ExtraTrees, 'LDA_KN': LDA_KN}
LDA_DF = pd.DataFrame.from_dict(res_LDA, orient='index', columns=['F1-mean'])

In [129]:
pd.concat([NMF_DF, SVD_DF, LDA_DF]).sort_values('F1-mean', ascending=True)

,F1-mean
SVD_ExtraTrees,0.21
NMF_ExtraTrees,0.26
LDA_ExtraTrees,0.36
SVD_RF,0.42
LDA_RF,0.42
NMF_KN,0.43
NMF_RF,0.45
SVD_KN,0.46
NMF_SGD,0.51
LDA_KN,0.51
